In [1]:
import os, pandas as pd,numpy as np
path_future = "C:\\PICKLE\\Nifty Future"
path_options = "C:\\PICKLE\\Nifty Options"

In [167]:
import warnings
warnings.filterwarnings('ignore')

In [247]:
# List all files in the Nifty Future folder
future_files = [os.path.join(path_future, f) for f in os.listdir(path_future) if os.path.isfile(os.path.join(path_future, f))]
future_files = [f for f in future_files if "2024" in f]
# List all files in the Nifty Options folder
option_files = [os.path.join(path_options, f) for f in os.listdir(path_options) if os.path.isfile(os.path.join(path_options, f))]
option_files = [f for f in option_files if "2024" in f]

In [ ]:
result_b120 = {}
result_decey={}

counter_b120 = 0
counter_dk = 0

base_dir = "nifty"
b120_dir = os.path.join(base_dir, "b120")
os.makedirs(b120_dir, exist_ok=True)

dk = os.path.join(base_dir, "decey")
os.makedirs(dk, exist_ok=True)

for nf, no in zip(future_files, option_files):

    nft = pd.read_pickle(nf).set_index('date_time')
    opt = pd.read_pickle(no).set_index('date_time')

    nft.index = pd.to_datetime(nft.index)
    opt.index = pd.to_datetime(opt.index)

    current_date = nft.index[0].date()
    print(f"Processing date: {current_date}")

    entry_time = [pd.to_datetime(f"{current_date} 09:20:00"),
                  pd.to_datetime(f"{current_date} 09:40:00")]
    exit_time = pd.to_datetime(f"{current_date} 15:25:00")


    for et in entry_time:
        if et not in opt.index:
            print(f"Entry time {et} not found in future data index.")
            continue
        if et in nft.index and et in opt.index:
            sl = [20,40,40]
            ut_sl = [30,40,50]
            for sl in sl:
                for utsl in ut_sl:
                    key = f"trade_{counter_b120}"
                    res = b120(nft, opt, sl, utsl, et, exit_time)
                    if res is not None:
                        result_b120[key] = res
                        counter_b120 += 1
                    # print("Current Results:", results)



    
    print(f"Completed processing for {os.path.basename(nf)}")


    for et in entry_time:
        if et in nft.index and et in opt.index:
            dk = [10,20]
            s_l = [20,30]
            for dk in dk:
                for sl in s_l:
                    entry_close = nft.loc[et, 'close']
                    fut_close_entry = round(entry_close / 100) * 100
                    ce_strike = f"{int(fut_close_entry)}CE"
                    pe_strike = f"{int(fut_close_entry)}PE"
                    ce_data = opt[(opt['scrip'] == ce_strike) ]
                    pe_data = opt[(opt['scrip'] == pe_strike) ]
                    

                    key = f"trade_{counter_dk}"
                    res = decey(ce_data, et, exit_time, entry_close, dk, sl, option="call")
                    if res is not None:
                        result_decey[key] = res
                        counter_dk += 1
                    # print("Current Results:", results)

                    key = f"trade_{counter_dk}"
                    res = decey(pe_data, et, exit_time, entry_close, dk, sl, option="put")
                    if res is not None:
                        result_decey[key] = res
                        counter_dk += 1
                    # print("Current Results:", results)

    
    print(f"Completed processing for {os.path.basename(nf)}")

results_df = pd.DataFrame.from_dict(result_b120, orient='index')
results_decey_df = pd.DataFrame.from_dict(result_decey, orient='index')

results_df.to_csv(f"results_b120.csv")
results_decey_df.to_csv(f"results_decey.csv")


Processing date: 2024-01-01
Completed processing for 2024-01-01_nifty_future.pkl
Completed processing for 2024-01-01_nifty_future.pkl
Processing date: 2024-01-02
Completed processing for 2024-01-02_nifty_future.pkl
Completed processing for 2024-01-02_nifty_future.pkl
Processing date: 2024-01-03
Completed processing for 2024-01-03_nifty_future.pkl
Completed processing for 2024-01-03_nifty_future.pkl
Processing date: 2024-01-04
Completed processing for 2024-01-04_nifty_future.pkl
Completed processing for 2024-01-04_nifty_future.pkl
Processing date: 2024-01-05
Completed processing for 2024-01-05_nifty_future.pkl
Completed processing for 2024-01-05_nifty_future.pkl
Processing date: 2024-01-08
Completed processing for 2024-01-08_nifty_future.pkl
Completed processing for 2024-01-08_nifty_future.pkl
Processing date: 2024-01-09
Completed processing for 2024-01-09_nifty_future.pkl
Completed processing for 2024-01-09_nifty_future.pkl
Processing date: 2024-01-10
Completed processing for 2024-01-1

In [155]:
def b120(fut:pd.DataFrame, opt:pd.DataFrame, sl:int, ut_sl:int, entry_time:pd.Timestamp, exit_time:pd.Timestamp) -> dict:
    # Find future close at entry and round to nearest 100
    fut_close_entry = fut.loc[entry_time, 'close']
    # print(fut_close_entry)

    # if isinstance(fut_close_entry, pd.Series) or isinstance(fut_close_entry, np.ndarray):
    #     fut_close_entry = fut_close_entry.iloc[0]  # Take the first value
        
    fut_close_entry = round(fut_close_entry / 100) * 100

    ce_strike = f"{int(fut_close_entry)}CE"
    pe_strike = f"{int(fut_close_entry)}PE"

    # First filter by strike price, then by time
    ce_data = opt[opt['scrip'] == ce_strike]
    pe_data = opt[opt['scrip'] == pe_strike]
    
    # Now slice by time
    ce_data = ce_data.loc[entry_time:]
    pe_data = pe_data.loc[entry_time:]

    ce_price = ce_data.loc[entry_time, 'close']
    pe_price = pe_data.loc[entry_time, 'close']

    ce_sl_price = ce_price + ((ce_price * sl) / 100)
    pe_sl_price = pe_price + ((pe_price * sl) / 100)

    ce_hit_time = ce_data[ce_data['high'] >= ce_sl_price].index.min()
    pe_hit_time = pe_data[pe_data['high'] >= pe_sl_price].index.min()

    ce_pnl = None
    pe_pnl = None

    try:
        if pd.notna(ce_hit_time) and pd.notna(pe_hit_time):
            if ce_hit_time <= pe_hit_time:
                first_sl = 'ce'
                first_hit_time = ce_hit_time
            else:
                first_sl = 'pe'
                first_hit_time = pe_hit_time
        elif pd.notna(ce_hit_time):
            first_sl = 'ce'
            first_hit_time = ce_hit_time
        elif pd.notna(pe_hit_time):
            first_sl = 'pe'
            first_hit_time = pe_hit_time
        else:
            first_sl = None
            first_hit_time = None

        if first_sl == 'ce':
            ce_pnl = ce_price - ce_sl_price
            pe_ut_sl_price = pe_price + ((pe_price * ut_sl) / 100)
            pe_ut_hit_time = pe_data.loc[first_hit_time:][pe_data['high'] >= pe_ut_sl_price].index.min()

            if pd.notna(pe_ut_hit_time):
                pe_exit_price = pe_data.loc[pe_ut_hit_time, 'close']
            else:
                pe_exit_price = pe_data.loc[exit_time, 'close']
            pe_pnl = pe_exit_price - pe_price

        elif first_sl == 'pe':
            pe_pnl = pe_price - pe_sl_price
            ce_ut_sl_price = ce_price + ((ce_price * ut_sl) / 100)
            ce_ut_hit_time = ce_data.loc[first_hit_time:][ce_data['high'] >= ce_ut_sl_price].index.min()

            if pd.notna(ce_ut_hit_time):
                ce_exit_price = ce_data.loc[ce_ut_hit_time, 'close']
            else:
                ce_exit_price = ce_data.loc[exit_time, 'close']
            ce_pnl = ce_exit_price - ce_price

        else:
            ce_exit_price = ce_data.loc[exit_time, 'close']
            pe_exit_price = pe_data.loc[exit_time, 'close']
            ce_pnl = ce_exit_price - ce_price
            pe_pnl = pe_exit_price - pe_price

    except Exception as e:
        print("Error in PnL calculation:", e)
        return None

    total_pnl = (ce_pnl if ce_pnl is not None else 0) + (pe_pnl if pe_pnl is not None else 0)

    return {"entry_time":entry_time,
            "exit_time":exit_time,
            "ce_price":ce_price,
            "pe_price":pe_price,
            "ce_hit_time": ce_hit_time,
            "pe_hit_time":pe_hit_time,
            "sl": sl, 
            "utsl": utsl,
            "ce_pnl":ce_pnl, 
            "pe_pnl":pe_pnl,
            "total_pnl":total_pnl
            }


In [165]:
def decey(data, entry_time, exit_time, entry_price, dk, sl, option=None) -> dict:
    """
    Vectorized trade strategy for CE/PE options (SELL side).
    """
    # Initial target and stoploss
    dk_price = entry_price * (1 - dk / 100)
    sl_price = dk_price * (1 + sl / 100)

    # Slice data for the trading window
    data_window = data.loc[entry_time:exit_time]

    # Find first time target (dk) is hit
    target_hit_time = data[data['low'] <= dk_price].index.min()


    if pd.notna(target_hit_time) :
    
        sl_data = data_window[data_window.index >= target_hit_time]
        sl_hit_time = sl_data[sl_data['high'] >= sl_price].index.min()

        if (pd.isna(sl_hit_time) ):
            exit_time = sl_data.index.max()
            exit_price = sl_data.loc[exit_time, 'close']
            trade_type = 'target'
            pnl =  dk_price - exit_price  # For sell side
        else:
            exit_time = sl_hit_time
            exit_price =  sl_price
            trade_type = 'stoploss'
            pnl = dk_price - exit_price  # For sell side

    else :

        target_hit_time = None
        sl_hit_time = None
        pnl = 0
        exit_time = data.index.max()

    return {
        "entry_time":entry_time,
        "exit_time": exit_time,
        "entry_price": entry_price,
        "dk_price": dk_price,
        "sl_price": sl_price,
        "target_hit_time": target_hit_time,
        "sl_hit_time": sl_hit_time,
        "trade_type": trade_type if pd.notna(target_hit_time) else 'no_trade',
        "pnl": pnl,
        "option": option,
        "trade_type": trade_type if pd.notna(target_hit_time) else 'no_trade'
    }


